In [1]:
from skimage import data, color, util
from skimage.restoration import denoise_tv_chambolle
from skimage.feature import hog

In [2]:
def task(image):
    # Apply some functions and return an image
    image = denoise_tv_chambolle(image[0][0], weight=0.1, multichannel=True)
    fd, hog_image = hog(color.rgb2gray(image), orientations=8,
                       pixels_per_cell=(16,16), cells_per_block=(1,1),
                       visualise=True)
    return hog_image

In [3]:
# Prepare images
hubble = data.hubble_deep_field()
width = 10
pics = util.view_as_windows(
    hubble, (width, hubble.shape[1], hubble.shape[2]), step=width)

In [4]:
# time a classic loop:
def classic_loop():
    for image in pics:
        task(image)
        
%timeit classic_loop()

1 loop, best of 3: 2.38 s per loop


In [5]:
# or a list comprehension:
def comprehension_loop():
    [task(image) for image in pics]
    
%timeit comprehension_loop()

1 loop, best of 3: 2.38 s per loop


In [7]:
# but using joblib we can parallelize loops:
# AND IT ACTUALLY SEEMS TO TAKE FOREVER ?????
from joblib import Parallel, delayed
def joblib_loop():
    Parallel(n_jobs=4)(delayed(task)(i) for i in pics)
    
%timeit joblib_loop()

1 loop, best of 3: 857 ms per loop
